In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
import numpy as np
import pandas as pd
import tensorflow as tf
from Tools.leica_tools import RawLoader
from Tools.db_tools import DbManager
from functools import partial
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from keras.api.models import Model
from keras.api.layers import Input, Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.api.losses import BinaryCrossentropy
from keras.api.metrics import BinaryAccuracy
from keras.api.optimizers import Adam
from keras.api.utils import plot_model

# KMeans detection

In [83]:
expID = 'NKIP_FA_070'
os.mkdir(os.path.join(os.getenv('EXP_DIR'), expID, 'KMeans'))

In [84]:
rawloader = RawLoader(expID)
dbm = DbManager()
droplet_df = rawloader.get_droplet_df()

In [85]:
ds, spec = dbm.get_dataset(expID, return_spec=True)
bins = 256
histograms = np.zeros((spec['n_frames'], bins))
globalIDs = np.zeros(spec['n_frames'])
for i, element in enumerate(ds.as_numpy_iterator()):
    globalIDs[i] = element['GlobalID']
    frame = element['frame']
    hist, bins = np.histogram(frame[:, :, 0].flatten(), bins=bins, range=(0, 65535), density=True)
    histograms[i, :] = hist / np.sum(hist)

2024-09-13 14:43:08.090525: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [90]:
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(histograms)
cluster_labels, counts = np.unique(clusters, return_counts=True)
for l, c in zip(cluster_labels, counts):
    print(f'Group {l} labeled with {c} droplets')
cluster_df =  pd.DataFrame(clusters, index=pd.Index(globalIDs, name='GlobalID').astype(int), columns=['cluster'])

Group 0 labeled with 23239 droplets
Group 1 labeled with 1763 droplets
Group 2 labeled with 21308 droplets
Group 3 labeled with 11892 droplets
Group 4 labeled with 9151 droplets
Group 5 labeled with 10754 droplets
Group 6 labeled with 16396 droplets
Group 7 labeled with 102 droplets


In [91]:

with PdfPages(os.path.join(os.getenv('EXP_DIR'), expID, 'KMeans', f'KMeans_{n_clusters}c.pdf')) as pdf:
    for c in cluster_labels:
        fig, axs = plt.subplots(figsize=(4,4), ncols=4, nrows=4)
        IDs = cluster_df.query(f'cluster == {c}').sample(16).index
        frames = dbm.filter_db(expID, IDs)[:, :, :, 0]
        for i, ax in enumerate(axs.flatten()):
            ax.imshow(frames[i]/65535, cmap='gray', vmin=0, vmax=1)
            ax.grid(False)
            ax.set_yticks([])
            ax.set_xticks([])
        fig.suptitle(f'Samples of cluster {c}', fontsize=15)
        pdf.savefig(fig)
        plt.close()

2024-09-13 14:46:42.735798: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-13 14:46:42.860892: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-13 14:46:42.993043: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-13 14:46:43.109358: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-13 14:46:43.225087: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-13 14:46:43.349648: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-13 14:46:43.470581: W tensorflow/core/framework/local_rendezvous.cc:404] L

In [92]:
cluster_df['outlier_KMeans'] = False
cluster_df.loc[cluster_df.query('cluster == 1 | cluster == 4 | cluster == 7').index, 'outlier_KMeans'] = True

In [93]:
rawloader.update_droplet_df(droplet_df.join(cluster_df))

# CNN Training

In [3]:
dbm = DbManager()

In [4]:
def outlier_class():
    inputs = Input(shape=(128,128,1), name='outlier_input')
    
    # Add convolutional layer
    conv1 = Conv2D(32, (3, 3), activation='relu', name='outlier_conv1')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2), name='outlier_pool1')(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', name='outlier_conv2')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2), name='outlier_pool2')(conv2)
    
    conv3 = Conv2D(128, (3, 3), activation='relu', name='outlier_conv3')(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2), name='outlier_pool3')(conv3)
    
    # Flatten the output from the convolutional layers
    flatten = Flatten(name='outlier_flatten')(pool3)
    
    # Add fully connected layers
    dense1 = Dense(512, activation='relu', name='outlier_dense1')(flatten)
    dropout1 = Dropout(0.5, name='outlier_dropout1')(dense1)  # Dropout rate of 0.5 (50% dropout rate)
    
    dense2 = Dense(256, activation='relu', name='outlier_dense2')(dropout1)
    dropout2 = Dropout(0.5, name='outlier_dropout2')(dense2)  # Dropout rate of 0.5 (50% dropout rate)
    
    dense3 = Dense(128, activation='relu', name='outlier_dense3')(dropout2)
    dropout3 = Dropout(0.5, name='outlier_dropout3')(dense3)  # Dropout rate of 0.5 (50% dropout rate)
    
    output = Dense(2, activation='softmax', name='outlier_output')(dropout3)  # Binary classification with sigmoid activation
    
    model = Model(inputs=inputs, outputs=output, name='outlier_model')

    return model

In [5]:
def prepare_data(element, outlier_data):
    globalID = element['GlobalID']
    expID = element['expID']
    element['outlier_input'] = tf.cast(element['frame'][:, :, tf.constant(0)], tf.float32) / 65535
    
    outputs = {}
    label = tf.py_function(lambda x, i: outlier_data.loc[(x.numpy().decode(), i.numpy()), 'outlier_KMeans'], [expID, globalID], tf.int64)
    label.set_shape(())
    label = tf.cast(tf.one_hot(label, 2), tf.int64)
    outputs['outlier_output'] = label
    return element, outputs

def build_dataset(expIDs):
    dataset = dbm.get_datasets(expIDs, shuffle=True)

    outlier_dfs = []
    for expID in expIDs:
        droplet_df = RawLoader(expID).get_droplet_df()
        outlier_df = droplet_df[['outlier_KMeans',]].copy()
        outlier_df.set_index(pd.MultiIndex.from_product([[expID], outlier_df.index]), inplace=True)  
        outlier_dfs.append(outlier_df)
    outlier_df = pd.concat(outlier_dfs)

    annotated_dataset = dataset.map(partial(prepare_data, outlier_data=outlier_df))
    return annotated_dataset

In [6]:
#expIDs = ['NKIP_FA_052','NKIP_FA_053', 'NKIP_FA_055', 'NKIP_FA_056'] # outlier_v2 datasets
expIDs = ['NKIP_FA_052','NKIP_FA_053', 'NKIP_FA_055', 'NKIP_FA_056', 'NKIP_FA_063', 'NKIP_FA_064', 'NKIP_FA_065', 'NKIP_FA_066','NKIP_FA_067','NKIP_FA_069','NKIP_FA_070'] # outlier_v3 dataset
dataset = build_dataset(expIDs)
validation_dataset = build_dataset(['NKIP_FA_051'])

In [101]:
n_elements = dataset.reduce(tf.constant(0), lambda a,b: a+1).numpy()
n_elements_val = validation_dataset.reduce(tf.constant(0), lambda a,b: a+1).numpy()
print(f'{n_elements} frames in train dataset')
print(f'{n_elements_val} frames in test dataset')

970996 frames in train dataset
126699 frames in test dataset


In [8]:
train_final = dataset.shuffle(15000).repeat(2).batch(32)
test_final = validation_dataset.repeat(2).batch(32)

In [9]:
model = outlier_class()

In [ ]:
model_arch = plot_model(model, to_file='outlier_exclusion.png', dpi=100)

In [10]:
model.compile(optimizer=Adam(),
              loss={'outlier_output': BinaryCrossentropy()},
              metrics={'outlier_output': BinaryAccuracy()})

In [11]:
model.fit(train_final, validation_data=test_final, batch_size=32, steps_per_epoch=30343, epochs=2, validation_steps=4701)

Epoch 1/2
30343/30343 ━━━━━━━━━━━━━━━━━━━━ 3623s 119ms/step - binary_accuracy: 0.9581 - loss: 0.1162 - val_binary_accuracy: 0.9546 - val_loss: 0.1143
Epoch 2/2
30343/30343 ━━━━━━━━━━━━━━━━━━━━ 3610s 119ms/step - binary_accuracy: 0.9733 - loss: 0.0731 - val_binary_accuracy: 0.9575 - val_loss: 0.1135


2024-09-13 21:00:55.376150: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/fauberma/mambaforge/envs/phenotyping/lib/python3.12/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [12]:
model.save(os.path.join(os.getenv('MODEL_DIR'),'outlier_exclusion', 'outlier_v3.h5'))

In [ ]:
# Define a function to normalize your data
def prepare_data(element, outlier_df):
    globalID = element['GlobalID']  # Assuming 'GlobalID' is a tensor
    expID = element['expID']
    frame = tf.cast(element['frame'][:, :, tf.constant(0)], tf.float32) / 65535

    #label = tf.py_function(lambda x: outlier_vec.get(x.numpy()), [globalID], tf.int64)
    label = tf.py_function(lambda x, i: outlier_df.loc[(x.numpy().decode(), i.numpy()), 'outlier_KMeans'], [expID, globalID], tf.int64)
    label.set_shape(())
    label = tf.cast(tf.one_hot(label, 2), tf.int64)
    element['outlier_output'] = label
    element['outlier_input'] = frame
    return element, element


In [ ]:
def get_data(expID):
    rawloader = RawLoader(expID)
    drop_register = rawloader.get_dropregister()
    outlier_df = drop_register[['outlier_KMeans',]].copy().reset_index()
    outlier_df['expID'] = expID
    outlier_df.set_index(['expID', 'GlobalID'], inplace=True)
    dataset = dbm.get_dataset(expID)
    return dataset, outlier_df

# Inference

In [ ]:
expID = 'AT_2024_007'
rawloader = RawLoader(expID)
drop_register = rawloader.get_dropregister()
label_dict = drop_register['outlier'].to_dict()

In [ ]:
dataset = dbm.get_dataset(expID)
normalized_dataset = dataset.map(partial(normalize, label_dict=label_dict)).batch(32)
y_predict_raw = model.predict(normalized_dataset)


In [ ]:
globalIDs = [e['GlobalID'] for e in dataset.as_numpy_iterator()]
y_predict = np.argmax(y_predict_raw,axis=1).astype(bool)
drop_register.loc[globalIDs, 'outlier'] = y_predict
rawloader.update_dropregister(drop_register)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
with PdfPages(os.path.join(rawloader.an_dir, 'outlier_summary.pdf')) as pdf:
    for outlier in [True, False]:
        fig, axs = plt.subplots(figsize=(8,8), ncols=8, nrows=8)
        subset = drop_register.query(f'outlier == {outlier}').copy()
        size = subset.index.size
        IDs = subset.sample(min(size,64)).index
        frames = dbm.filter_db(expID, IDs)[:, :, :, 0]
        for i, ax in enumerate(axs.flatten()):
            ax.imshow(frames[i]/65535, cmap='gray', vmin=0, vmax=1)
            ax.grid(False)
            ax.set_yticks([])
            ax.set_xticks([])
        if outlier:
            fig.suptitle(f'Samples of detected outliers ({size} in total)', fontsize=15)
        else:
            fig.suptitle(f'Samples of detected non-outliers ({size} in total)', fontsize=15)
        pdf.savefig(fig)
        plt.close()